# MONAI Deploy App SDKを使ったMedNIST分類器アプリのデプロイ (Prebuilt Model)

このチュートリアルでは、MONAI Deploy App SDKを使って学習済みのモデルを、推論を行うローカルプログラム、同じことを行うワークフロージョブ、Dockerコンテナによるワークフロー実行として実行可能なアーティファクトにパッケージングするプロセスをデモしています。

このチュートリアルでは、学習済みモデルを使用し、推論アプリケーションを実装・パッケージ化し、ローカルでアプリケーションを実行します。


## githubプロジェクトをクローンする（mainブランチの最新版のみ）


In [1]:
!git clone --branch main --depth 1 https://github.com/Project-MONAI/monai-deploy-app-sdk.git source \
 && rm -rf source/.git

Cloning into 'source'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (264/264), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 264 (delta 52), reused 104 (delta 14), pack-reused 0
Receiving objects: 100% (264/264), 791.21 KiB | 3.18 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
!ls source/examples/apps/mednist_classifier_monaideploy/

mednist_classifier_monaideploy.py


## monai-deploy-app-sdk パッケージをインストールします

In [3]:
!pip install --upgrade monai-deploy-app-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 9.0 MB/s 
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Uninstalling typeguard-2.7.1:
      Successfully uninstalled typeguard-2.7.1


## アプリに必要なパッケージのインストール

In [4]:
!pip install monai Pillow  # for MONAI transforms and Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 8.7 MB/s 


## Google Driveからmednist_classifier_data.zipをダウンロードと解凍を行う

In [5]:
# Download mednist_classifier_data.zip
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1yJ4P-xMNEfN6lIOq_u6x1eMAq1_MJu-E"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1yJ4P-xMNEfN6lIOq_u6x1eMAq1_MJu-E
To: /content/mednist_classifier_data.zip
100% 28.6M/28.6M [00:00<00:00, 68.9MB/s]


In [6]:
# After downloading mednist_classifier_data.zip from the web browser or using gdown,
!unzip -o "mednist_classifier_data.zip"

Archive:  mednist_classifier_data.zip
 extracting: classifier.zip          
 extracting: input/AbdomenCT_007000.jpeg  


## アプリのパッケージ化（MAP Dockerイメージの作成）

ローカルマシンにnvidia dockerがインストールされていることが前提です。

nvidia-docker2をインストールするには、https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#docker を参照してください。

`-l DEBUG` オプションを使用すると進捗を確認できる。

In [ ]:
!monai-deploy package "source/examples/apps/mednist_classifier_monaideploy/mednist_classifier_monaideploy.py" \
    --tag mednist_app:latest \
    --model classifier.zip

Building MONAI Application Package... /bin/sh: 1: docker: not found
Done


## dockerイメージと入力ファイルをローカルに置いてアプリを実行する

In [ ]:
!monai-deploy run mednist_app:latest "input" "output"

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "mednist_app:latest" is available...

Checking for MAP "mednist_app:latest" locally
"mednist_app:latest" found.

Reading MONAI App Package manifest...
 > export '/var/run/monai/export/' detected
--> Verifying if "nvidia-docker" is installed...

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 1, Operator ID: 2000b9d2-156f-4abd-8654-cf60219673ac)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 1, Operator ID: 13deb10c-dd13-4af5-8a05-a72c07406c05)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [ ]:
!cat output/output.json

"AbdomenCT"

## MONAI Deploy App SDKによるアプリケーションの実装とパッケージング

Torchscriptのモデル（classifier.zip）を元に、入力されたJpeg画像を処理し、予測（分類）結果をJSONファイル（output.json）として書き出すアプリを実装します。

推論アプリケーションでは、2つの演算子を定義します。

1. `LoadPILOperator` - 入力パスからJPEG画像をロードし、ロードされた画像オブジェクトを次のオペレータに渡します。
    - このオペレータは、*train_transforms* の`LoadImage(image_only=True)` 変換と同様の働きをしますが、扱う画像は1枚だけです。
    - **Input**: ファイルパス ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output**: メモリ上の画像オブジェクト ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
2. `MedNISTClassifierOperator` - 与えられた画像をMONAIの`Compose`クラスで前変換し、Torchscriptのモデル (`classifier.zip`)に送り、予測結果をJSONファイル(`output.json`)に書き出す。
    - プリトランスフォームは、3つのトランスフォームで構成されている。 -- `AddChannel`, `ScaleIntensity`, `EnsureType`.
    - **Input**: メモリ上の画像オブジェクト ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: 予測結果(`output.json`)が書き込まれるフォルダパス ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))

アプリケーションのワークフローは以下のようになる。

<img src="https://user-images.githubusercontent.com/1928522/133868503-46671f0a-7741-4f9d-aefa-83e95e9a5f84.png" alt="Workflow" style="width: 600px;margin-left:auto;margin-right:auto;"/>


### インポートの設定

必要なクラスやデコレータをインポートし、`MEDNIST_CLASSES`を定義します。

In [ ]:
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]

### Operatorクラスの作成

#### LoadPILOperator

In [ ]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """与えられた入力（DataPath）から画像を読み込み、出力（Image）にnumpy配列を設定します。"""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # 最初のファイルを取る

        image = PILImage.open(input_path)
        image = image.convert("L")  # グレースケール画像に変換する
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # numpyの配列でImage domainオブジェクトを生成します。
        op_output.set(output_image)

#### MedNISTClassifierOperator

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """与えられた画像を分類し、クラス名を返す。"""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # TorchScriptModel オブジェクトを取得します。

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # クラス名を取得する
        print(result)

        # 出力（フォルダ）パスを取得し、存在しない場合はフォルダを作成する
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # 結果を "output.json "に書き込む
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)

### アプリケーションクラスの作成

アプリケーションクラスは次のようなものです。

`Application`クラスを継承した`App`クラスが定義されています。

`LoadPILOperator` と`MedNISTClassifierOperator`は、 `App`の`compose()`メソッド内で`self.add_flow()`を使って接続されています。

In [ ]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)

### アプリをローカルで実行する

Jupyterノートブックでアプリを実行することができます。

In [ ]:
app = App()

In [ ]:
app.run(input="input/AbdomenCT_007000.jpeg", output="output", model="classifier.zip")

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 7041, Operator ID: 3aa42bbd-f8dd-4374-98ee-7b614979e75a)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 7041, Operator ID: 7ee7dd5e-c042-4245-bb75-15ff064bd838)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [ ]:
!cat output/output.json

"AbdomenCT"

Jupyter notebook内でアプリケーションの検証ができたら、上記のコードを連結して、アプリケーション全体をファイル(`mednist_classifier_monaideploy.py`)として記述し、以下の行を追加します。

```python
if __name__ == "__main__":
    App(do_run=True)
```

上記の行は `python` インタープリタを使ってアプリケーションコードを実行するために必要なものです。

In [ ]:
%%writefile mednist_classifier_monaideploy.py

# Copyright 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import monai.deploy.core as md  # 'md'はMONAI Deployの略です（'core'でも可）
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """与えられた入力（DataPath）から画像を読み込み、出力（Image）にnumpy配列を設定します。"""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # 最初のファイルを取る

        image = PILImage.open(input_path)
        image = image.convert("L")  # グレースケール画像に変換する
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # numpyの配列でImage domainオブジェクトを生成します。
        op_output.set(output_image)


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """与えられた画像を分類し、クラス名を返す。"""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  #TorchScriptModel オブジェクトを取得します。

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # クラス名を取得する
        print(result)

        # 出力（フォルダ）パスを取得し、存在しない場合はフォルダを作成する。
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # 結果を "output.json "に書き込む
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)


@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """MedNIST分類器のアプリケーションクラス"""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)


if __name__ == "__main__":
    App(do_run=True)

Writing mednist_classifier_monaideploy.py


今回は、コマンドラインでアプリを実行してみましょう

In [ ]:
!python "mednist_classifier_monaideploy.py" -i "input/AbdomenCT_007000.jpeg" -o output -m "classifier.zip"

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 8412, Operator ID: 631a82bf-c90e-4217-a17c-831b2c74bc50)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 8412, Operator ID: a8fe1121-68bb-463f-bf1c-beff38d4fe86)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



上記のコマンドは、以下のコマンドラインと同じです。

In [ ]:
!monai-deploy exec "mednist_classifier_monaideploy.py" -i "input/AbdomenCT_007000.jpeg" -o output -m "classifier.zip"

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 8453, Operator ID: 7dec2a01-6d18-4104-b250-5b93d663ba4f)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 8453, Operator ID: 5e83dd80-5b19-4c78-9382-3d181640b80c)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [ ]:
!cat output/output.json

"AbdomenCT"